In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline

# --- Setup and Initialization (Placeholders for real model/data) ---

# Load Preprocessing and Sentiment Model (Placeholder)
# In a real scenario, you would use data_preprocessing functions here
# and load the actual sentiment_model.pkl from Part 2.
try:
    # Load the model saved in Part 2
    sentiment_pipeline = joblib.load('sentiment_model.pkl')
    # Use a basic summarizer for the demo
    summarizer_pipe = pipeline("summarization", model="facebook/bart-large-cnn")
    IS_MODEL_LOADED = True
except:
    IS_MODEL_LOADED = False
    
    # Fallback/Mock implementation if model files are missing
    class MockModel:
        def predict(self, text):
            # Simple mock prediction logic
            if 'great' in text or 'love' in text: return 'Positive'
            if 'slow' in text or 'bad' in text: return 'Negative'
            return 'Neutral'
    
    sentiment_pipeline = MockModel()

# Mock data for initial display and charts
if 'df' not in st.session_state:
    st.session_state.df = pd.DataFrame({
        'feedback_text': [
            "The service was fast and the product is excellent. Highly recommend!",
            "I waited three weeks for delivery, absolutely unacceptable delay.",
            "It works as described, no major complaints but nothing special.",
            "Great customer support, they fixed my billing issue in minutes.",
            "The app crashes every time I try to log in. Extremely frustrating and a bad design."
        ],
        'sentiment': ['Positive', 'Negative', 'Neutral', 'Positive', 'Negative']
    })

# --- Main Functions ---

def analyze_and_update(df):
    """Analyzes the loaded dataframe and updates sentiments."""
    st.session_state.df['sentiment'] = st.session_state.df['feedback_text'].apply(sentiment_pipeline.predict)
    return st.session_state.df

def generate_summaries(text):
    """Generates short and detailed summaries for a single text."""
    try:
        short = summarizer_pipe(text, max_length=30, min_length=10, do_sample=False)[0]['summary_text']
        detailed = summarizer_pipe(text, max_length=60, min_length=30, do_sample=False)[0]['summary_text']
        return short, detailed
    except Exception as e:
        return f"Error: {e}", "Could not generate detailed summary."


# --- Streamlit UI ---

st.set_page_config(layout="wide", page_title="Intelligent Customer Feedback Analysis")

st.title("🤖 Intelligent Customer Feedback Analysis System")
st.markdown("Analyze, summarize, and predict customer sentiment from raw feedback data.")

if not IS_MODEL_LOADED:
    st.warning("Sentiment model ('sentiment_model.pkl') not found. Using mock prediction logic.")

# --- File Upload Section (Part 5) ---

st.header("1. Upload & Analyze Feedback Data")
uploaded_file = st.file_uploader("Choose a CSV or JSON file", type=['csv', 'json'])

if uploaded_file is not None:
    try:
        if uploaded_file.name.endswith('.csv'):
            new_df = pd.read_csv(uploaded_file)
        elif uploaded_file.name.endswith('.json'):
            new_df = pd.read_json(uploaded_file)
        
        # Ensure the column name matches
        if 'feedback_text' not in new_df.columns:
            st.error("Uploaded file must contain a column named 'feedback_text'.")
        else:
            st.session_state.df = new_df[['feedback_text']]
            st.success("File uploaded successfully! Click 'Run Sentiment Analysis' below.")
    except Exception as e:
        st.error(f"Error reading file: {e}")

if st.button("Run Sentiment Analysis", use_container_width=True):
    st.session_state.df = analyze_and_update(st.session_state.df)
    st.success("Sentiment analysis complete!")

st.markdown("---")

# --- Sentiment Visualization (Part 5) ---

st.header("2. Sentiment Distribution & Insights")

if 'sentiment' in st.session_state.df.columns:
    col1, col2 = st.columns([1, 2])
    
    # 2.1 Sentiment Breakdown Chart
    with col1:
        st.subheader("Sentiment Breakdown")
        sentiment_counts = st.session_state.df['sentiment'].value_counts()
        
        fig, ax = plt.subplots(figsize=(6, 6))
        sentiment_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90, ax=ax, 
                              colors=['#4CAF50', '#FFC107', '#F44336'])
        ax.set_ylabel('')
        st.pyplot(fig)
        
    # 2.2 Table View
    with col2:
        st.subheader("Raw Data Sample")
        st.dataframe(st.session_state.df[['feedback_text', 'sentiment']].head(10), height=300)
        
    # 2.3 Predictive Placeholder (Part 4)
    st.markdown("### Predictive Trend (Mock Data for Next Month)")
    st.info("Based on time-series analysis (Prophet/ARIMA), the predicted Customer Satisfaction Score for next month is **-0.12 (Slightly Negative)**, driven by recurring complaints about 'slow response time'.")

st.markdown("---")

# --- Summarization Interface (Part 5) ---

st.header("3. Text Summarization")

# Find a long piece of feedback to summarize
long_feedback = st.session_state.df['feedback_text'].iloc[0] if not st.session_state.df.empty else "No text loaded."

# Allow user to input or select text
summarize_text = st.text_area(
    "Enter a long feedback entry to summarize:", 
    value=long_feedback,
    height=150
)

if st.button("Generate Summaries", use_container_width=True) and summarize_text:
    short_summary, detailed_summary = generate_summaries(summarize_text)
    
    st.markdown("#### Short Summary (Key Takeaway)")
    st.success(short_summary)
    
    st.markdown("#### Detailed Summary (For deeper understanding)")
    st.info(detailed_summary)
